In [2]:
import pandas as pd
import io
import os
from datetime import date,datetime,timedelta
import collections
from pathlib import Path
collections.Callable=collections.abc.Callable
import numpy as np


In [3]:
os.chdir=(r"F:\Spectrum Auction\spectrum-auctions")
print(os.getcwd())

f:\Spectrum Auction\spectrum-auctions


In [5]:
service_area_dict = {
'Andhra Pradesh': 'AP',
'Assam':'AS',
'Bihar':'BH',
'Delhi': 'DEL',
'Gujarat':'GU',
'Haryana':'HA',
'Himachal Pradesh': 'HP',
'Jammu & Kashmir': 'JK',
'Karnataka' : 'KA',
'Kerala': 'KE',
'Kolkata': 'KOL',
'Madhya Pradesh' : 'MP',
'Maharashtra': 'MA',
'Mumbai':'MU',
'North East' : 'NE',
'Odisha': 'OR',
'Punjab': 'PU',
'Rajasthan': 'RA',
'Tamil Nadu' : 'TN',
'Uttar Pradesh (East)' : 'UPE',
'Uttar Pradesh (West)' : 'UPW',
'West Bengal': 'WB'
            }

band_dict={'900(MHz)':900,
           '2100(MHz)': 2100,
           '2500(MHz)': 2500,
           '1800(MHz)': 1800}
block_size_dict={
    1800:0.2, 900:0.2,2100:5,2500:10}
bidder_dict={'Vodafone Idea Limited': 'Vi',
             'Bharti Airtel Limited': 'Airtel',
             'Reliance Jio Infocomm Limited': 'Jio'}


In [6]:
df=pd.read_excel("Auction_Provisional_FAR.xlsx",engine='openpyxl',sheet_name="Sheet1")


In [7]:
df=df.dropna(axis=1,how='all').dropna(axis=0,how='all').ffill().ffill(axis=1)

In [8]:
df=df.replace(service_area_dict).replace(band_dict).replace(bidder_dict)

In [9]:
df = df[df['Unnamed: 4'].str.match(r'^\d*\.?\d*$', na=True)]

In [10]:
df.columns=["LSA","Band","Winning Price per block", "Bidder", "Uplink Start","Uplink Stop","Downlink Start","Downlink Stop"]

In [11]:
set(df["Bidder"])

{'Airtel', 'Jio', 'Vi'}

In [12]:
new_row = {
    'LSA': 'BH',
    'Band': 2500,
    'Winning Price per block': 150.000,
    'Bidder': 'Vi',
    'Uplink Start': 2545,
    'Uplink Stop': 2555,
    'Downlink Start': None,  
    'Downlink Stop': None,    
    'Block Size': 10
}
df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

In [13]:
df=df.reset_index(drop=True)

In [14]:
df["Block Size"]=[block_size_dict[x] for x in df["Band"]]

In [15]:
df=df.replace(r'\*','',regex=True)

In [16]:
df["MHz Won"] =df["Uplink Stop"].astype(float)-df["Uplink Start"].astype(float)

In [17]:
df["Bidder Outflow"]= (df["MHz Won"]/df["Block Size"])*df["Winning Price per block"].astype(float)

In [18]:
df

,LSA,Band,Winning Price per block,Bidder,Uplink Start,Uplink Stop,Downlink Start,Downlink Stop,Block Size,MHz Won,Bidder Outflow
0,AP,900,57.6,Vi,908.3,908.5,953.3,953.5,0.2,0.2,57.6
1,AP,900,57.6,Vi,908.1,908.3,953.1,953.3,0.2,0.2,57.6
2,AP,900,57.6,Vi,907.9,908.1,952.9,953.1,0.2,0.2,57.6
3,AP,900,57.6,Vi,907.7,907.9,952.7,952.9,0.2,0.2,57.6
4,AP,900,57.6,Vi,907.5,907.7,952.5,952.7,0.2,0.2,57.6
...,...,...,...,...,...,...,...,...,...,...,...
557,AS,2100,120.0,Airtel,1949.0,1954.0,2139.0,2144.0,5.0,5.0,120.0
558,JK,2100,40.0,Airtel,1974.0,1979.0,2164.0,2169.0,5.0,5.0,40.0
559,NE,2100,25.0,Airtel,1969.0,1974.0,2159.0,2164.0,5.0,5.0,25.0
560,UPW,2100,360.0,Airtel,1949.0,1954.0,2139.0,2144.0,5.0,5.0,360.0


In [19]:
df.groupby(["Bidder"])["Bidder Outflow"].sum()

Bidder
Airtel    6856.766
Jio        973.628
Vi        3510.400
Name: Bidder Outflow, dtype: float64

In [20]:
gov_revenue=df.groupby(["Bidder"])["Bidder Outflow"].sum().sum()
print(gov_revenue)

11340.794000000002


In [21]:
df.groupby(["LSA"])["MHz Won"].sum().sum()

141.40000000000032

In [22]:
total_mhz_won =df.groupby(["LSA"])["MHz Won"].sum().sum()
print(total_mhz_won)

141.40000000000032


In [23]:
df.groupby(['LSA','Band', 'Bidder'])['MHz Won'].sum()

LSA  Band  Bidder
AP   900   Vi         2.4
AS   900   Airtel     5.0
     1800  Airtel     4.0
     2100  Airtel     5.0
BH   900   Airtel     7.0
     1800  Airtel     5.2
           Jio        5.0
     2500  Vi        10.0
HA   1800  Airtel     5.0
JK   900   Airtel    10.0
     2100  Airtel     5.0
KA   900   Vi         2.2
KOL  900   Vi         0.2
     1800  Airtel     5.0
MP   1800  Vi         1.2
NE   900   Airtel     1.0
     2100  Airtel     5.0
OR   900   Airtel     5.0
     1800  Airtel     1.0
PU   900   Vi         1.2
     1800  Airtel     5.0
RA   900   Airtel     4.0
           Vi         0.4
     1800  Airtel     5.0
TN   900   Vi         2.4
UPE  900   Airtel     5.0
           Vi         1.2
     1800  Airtel     1.0
UPW  900   Vi         5.0
     2100  Airtel     5.0
WB   900   Airtel     5.0
           Vi         3.8
     1800  Airtel     3.8
           Jio        9.4
Name: MHz Won, dtype: float64

In [24]:
df.groupby(['LSA','Band', 'Bidder'])['Bidder Outflow'].sum()

LSA  Band  Bidder
AP   900   Vi         691.200
AS   900   Airtel     315.000
     1800  Airtel     128.000
     2100  Airtel     120.000
BH   900   Airtel    1029.000
     1800  Airtel     437.060
           Jio        420.250
     2500  Vi         150.000
HA   1800  Airtel     205.000
JK   900   Airtel     180.000
     2100  Airtel      40.000
KA   900   Vi         448.800
KOL  900   Vi          30.600
     1800  Airtel     545.000
MP   1800  Vi         118.800
NE   900   Airtel      16.000
     2100  Airtel      25.000
OR   900   Airtel     320.000
     1800  Airtel      39.000
PU   900   Vi         124.800
     1800  Airtel     305.000
RA   900   Airtel     540.000
           Vi          54.000
     1800  Airtel     420.000
TN   900   Vi         532.800
UPE  900   Airtel     930.000
           Vi         223.200
     1800  Airtel     184.000
UPW  900   Vi         760.000
     2100  Airtel     360.000
WB   900   Airtel     495.000
           Vi         376.200
     1800  Airtel     

In [25]:
total_rev= df.groupby(['LSA','Band', 'Bidder'])['Bidder Outflow'].sum().sum()
print(total_rev)

11340.794000000005
